In [1020]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import kss
import re
import networkx as nx
import warnings
warnings.filterwarnings("ignore")

In [783]:
#Mecab 객체 생성
mecab = Mecab()

In [997]:
# df 불러오기
df = pd.read_csv("./test.csv", header=None)
df.columns = ['article']

In [998]:
df = df.iloc[0:30]

In [999]:
# 중복 제거
df = df.drop_duplicates('article')
df.reset_index(inplace=True,drop=True)

In [1000]:
# kss 모듈로 문장화
temp = []
for i in range(len(df)):
    temp.append(kss.split_sentences(df['article'][i]))
df['sentence'] = temp

In [1001]:
#각 문장의 길이가 20 이하인건 삭제
for i in range(len(df)):  
    for j in range(len(df['sentence'][i])-1,-1,-1):
        if len(df['sentence'][i][j]) <= 20:
            df['sentence'][i].remove(df['sentence'][i][j])

In [1002]:
#문장 전체 길이가 10 이하는 제거
for i in range(len(df)):
    if len(df['sentence'][i]) <= 10:
        df.drop(i, inplace = True)
        
df.reset_index(inplace=True,drop=True)

In [1003]:
#stopwords list화
temp = pd.read_csv("./stop_words.csv",header = None, usecols = [0], names = ['stopwords'])                                                
stopwords = []
for i in range(temp.shape[0]):
    stopwords.append(temp['stopwords'][i])

In [1004]:
#Mecab을 활용한 tokenize 및 불용어 제거
result = []
for i in range(len(df)):
    token_list = []
    for sentence in df['sentence'][i]:
        sentence = re.sub('[-=+,#/\?©:^$©@*\"※~&·%｜!』\\‘◇”|\(\)\[\]\<\>`\'’“…》]', '', sentence)
        tokenized = mecab.morphs(sentence)
        for j in range(len(tokenized)-1,-1,-1):
            if tokenized[j] in stopwords:
                tokenized.remove(tokenized[j])
        token_list.append(tokenized)
    result.append(token_list)
df['token'] = result

In [1005]:
df

,article,sentence,token
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명..."
1,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ..."
2,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,..."
3,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ..."
4,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3..."
5,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ..."
6,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하..."
7,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ...."
8,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,..."


In [1006]:
# Word2Vec 모델링
# news_tokens = df['tokenized'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8
model = Word2Vec.load('word2vec.model')

# model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [1007]:
#벡터화 작업
def vectorize(tokens):
    result = []
    for token in tokens:
        vectors = []
        for sentence in token:
            init_v = np.array([0.0]*v_dimension)
            count = 0
            for word in sentence:
                word_vectors = model.wv
                if word in word_vectors.vocab:
                    count +=1
                    v = model.wv[word]
                    init_v = init_v + v
            init_v = init_v / count    
            vectors.append(init_v)
        result.append(vectors)
    
    frame = {'article' : df['article'], 'sentence': df['sentence'], 'token': df['token'], 'vector': result }
    result = pd.DataFrame(frame) 
    
    return result

In [1008]:
df = vectorize(df['token'])

In [1009]:
df

,article,sentence,token,vector
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.1004460559473658, -0.048762121123774926, ..."
1,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ...","[[0.7504085465148091, -0.40503870490938426, 0...."
2,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,...","[[-0.03843088780922699, 0.07523332014014679, 0..."
3,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ...","[[0.19445288507267833, -0.0103965991147561, 0...."
4,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3...","[[0.635544136604842, 0.08541283003218911, 0.83..."
5,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ...","[[0.24626479188308997, -0.28111700323300765, 0..."
6,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하...","[[0.4652947559106079, -0.12442229184406725, 0...."
7,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ....","[[0.7028303413905881, -0.12219022561542013, 0...."
8,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,...","[[0.3041603645803977, -0.20886570956481962, 0...."


In [1010]:
#코사인유사도 매트릭스
def similarity_matrix(vectors):
    sim_mat = np.zeros([len(vectors), len(vectors)])
    for i in range(len(vectors)):
        for j in range(len(vectors)):
            sim_mat[i][j] = cosine_similarity(vectors[i].reshape(1, v_dimension), vectors[j].reshape(1, v_dimension))[0,0]
    return sim_mat


In [1011]:
df['SimMatrix'] = df['vector'].apply(similarity_matrix)

In [1012]:
df

,article,sentence,token,vector,SimMatrix
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.1004460559473658, -0.048762121123774926, ...","[[1.0, 0.8390463844682985, 0.38402217976241126..."
1,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ...","[[0.7504085465148091, -0.40503870490938426, 0....","[[1.0, 0.6849624598007038, 0.9491063294782923,..."
2,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,...","[[-0.03843088780922699, 0.07523332014014679, 0...","[[1.0000000000000004, 0.5431313853998593, 0.29..."
3,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ...","[[0.19445288507267833, -0.0103965991147561, 0....","[[0.9999999999999999, 0.020770213584794636, -0..."
4,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3...","[[0.635544136604842, 0.08541283003218911, 0.83...","[[0.9999999999999997, 0.7537380932719095, 0.85..."
5,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ...","[[0.24626479188308997, -0.28111700323300765, 0...","[[1.0000000000000004, 0.5223812823305478, 0.53..."
6,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하...","[[0.4652947559106079, -0.12442229184406725, 0....","[[1.0000000000000002, 0.3847769690682543, 0.64..."
7,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ....","[[0.7028303413905881, -0.12219022561542013, 0....","[[0.9999999999999997, 0.6869661633995391, 0.68..."
8,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,...","[[0.3041603645803977, -0.20886570956481962, 0....","[[0.9999999999999997, 0.34999870105711894, 0.8..."


In [1013]:
#코사인 유사도를 활용해서 각 문장마다 점수 매기기
#pagerank 사용
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

In [1014]:
df['score'] = df['SimMatrix'].apply(calculate_score)

In [1015]:
df

,article,sentence,token,vector,SimMatrix,score
0,요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7...,[요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 ...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.1004460559473658, -0.048762121123774926, ...","[[1.0, 0.8390463844682985, 0.38402217976241126...","{0: 0.01795871228825599, 1: 0.0194407872547084..."
1,금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 첫...,[금감원 2018년 11월 이후 투자자 전액 배상 결정 금융투자상품 분쟁조정 사상 ...,"[[금감원, 2018, 11, 이후, 투자자, 전액, 배상, 결정, 금융, 투자, ...","[[0.7504085465148091, -0.40503870490938426, 0....","[[1.0, 0.6849624598007038, 0.9491063294782923,...","{0: 0.08284668298683674, 1: 0.0785451733839945..."
2,롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 있...,[롯데마트와 홈플러스도 침방울 차단 마스크 판매 본격화 대부분 편의점에서도 살 수 ...,"[[롯데마트, 홈플러스, 도, 침방울, 차단, 마스크, 판매, 본격, 화, 대부분,...","[[-0.03843088780922699, 0.07523332014014679, 0...","[[1.0000000000000004, 0.5431313853998593, 0.29...","{0: 0.08138790359620644, 1: 0.0553330965831385..."
3,머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요제 중...,"[머니투데이 문병환 기자 머니투데이와 공연 문화허브 M터치 가 펼치는 4대 가요, ...","[[머니, 투데이, 문, 병환, 기자, 머니, 투데이, 공연, 문화, 허브, M, ...","[[0.19445288507267833, -0.0103965991147561, 0....","[[0.9999999999999999, 0.020770213584794636, -0...","{0: 0.02732887650814984, 1: 0.0510182957113229..."
4,스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 위...,[스포츠경향 정세균 국무총리와 노사정 대표들이 1일 오전 10시30분 ‘코로나19 ...,"[[스포츠, 경향, 정세균, 국무총리, 노사정, 대표, 1, 오전, 10, 시, 3...","[[0.635544136604842, 0.08541283003218911, 0.83...","[[0.9999999999999997, 0.7537380932719095, 0.85...","{0: 0.029298134608698213, 1: 0.031697616647051..."
5,뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미국...,[뉴욕 연합뉴스 오진우 연합인포맥스 특파원 뉴욕증시에서 주요 지수는 1일 양호한 미...,"[[뉴욕, 연합뉴스, 오진우, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 주요, ...","[[0.24626479188308997, -0.28111700323300765, 0...","[[1.0000000000000004, 0.5223812823305478, 0.53...","{0: 0.03955025430747894, 1: 0.0345649053063711..."
6,코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산한...,[코로나19거 전 세계로 확산되는 가운데 미국 뉴욕 맨하탄 웨스트빌리지 7번가 한산...,"[[코로나, 19, 거, 전, 세계, 확산, 되, 는, 가운데, 미국, 뉴욕, 맨하...","[[0.4652947559106079, -0.12442229184406725, 0....","[[1.0000000000000002, 0.3847769690682543, 0.64...","{0: 0.060645582370942946, 1: 0.057909049775027..."
7,앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다. 추가 규제를 피한...,"[앵커 정부가 6·17 부동산 대책을 내놓은 지 보름가량 됐습니다., 추가 규제를 ...","[[앵커, 정부, 617, 부동산, 대책, 내놓, 은, 지, 보름, 가량, 됐, ....","[[0.7028303413905881, -0.12219022561542013, 0....","[[0.9999999999999997, 0.6869661633995391, 0.68...","{0: 0.06874896041402666, 1: 0.0669202877319058..."
8,6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 아...,[6월 서울 주택 매매가 석달 만에 상승 경기는 0.68% 뛰어 ‘풍선효과’ 뚜렷 ...,"[[6, 서울, 주택, 매매, 석, 달, 만, 상승, 경기, 는, 0, ., 68,...","[[0.3041603645803977, -0.20886570956481962, 0....","[[0.9999999999999997, 0.34999870105711894, 0.8...","{0: 0.09490203110443626, 1: 0.0565733248059762..."


In [1016]:
# score가 높은 순으로 3개만 문장 뽑기
def ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s) 
                         for i,s in enumerate(sentences)), 
                                reverse=True)
    top_n_sentences = [sentence
                        for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

In [1017]:
df['summary'] = df.apply(lambda x: 
                            ranked_sentences(x.sentence, 
                            x.score), axis=1)

In [1018]:
temp = df[['article','summary']]

In [1019]:
for i in range(0, 2):
    print(i+1,'번 문서')
    print('원문 :',temp.loc[i].article)
    print('')
    print('요약 :',temp.loc[i].summary)
    print('')


1 번 문서
원문 : 요양보호사 광주 46번서 요양시설·교회로 전파…12명 추가 감염 서울 관악구서만 7명…동작구 소재 문창초 전수검사 실시 1일 오후 광주 동구 동명동 CCC 아가페실버센터 에서 질병관리본부 관계자들이 확진자 음압병실 이송 절차를 밟기 위해 센터에 들어가고 있다. 해당 센터는 코로나19 광주지역 46번 확진자가 요양보호사로 근무하는 곳이며 이날 센터 입소자 2명이 추가 확진 판정을 받았다.2020.7.1 뉴스1 © News1 황희규 기자 서울 뉴스1 김태환 기자 음상준 기자 이영성 기자 이형진 기자 신종 코로나바이러스 감염증 코로나19 일일 확진자가 1일 하루 동안 최소 35명 발생했다. 해외유입 4명을 제외한 31명이 지역에서 발생했다. 지역발생 감염자는 서울과 경기 등 수도권과 광주광역시에 집중됐다. 질병관리본부와 서울시 광주시 등 전국 17개 시·도 지자체에 따르면 이날 0시부터 오후 10시까지 발생한 신규 확진자는 광주 16명 서울 12명 경기 5명 대전 1명 충남 1명 등 최소 34명으로 나타났다. 특히 광주에선 감염경로가 파악이 되지 않은 요양보호사 확진자로부터 요양시설 입소환자 그가 다닌 교회 신도에 이르기까지 감염자가 무더기로 쏟아졌다. 서울 관악구에서는 감염경로를 알 수 없는 확진자만 7명이 나왔다. ◇광주 최소 16명…감염경로 모르는 46번서 12명 추가 감염 이날 광주 확진자 수는 16명 늘어난 72명을 기록했다. 감염경로를 알 수 없는 요양보호사 확진자로부터 시설입소 환자 3명과 그가 다닌 교회 신도 9명이 감염됐다. 광주 광륵사 관련 추가 감염자 3명도 발생했고 감염경로를 모르는 요양보호사 확진자 1명이 더 나왔다. 광주 북구 오치동 광주사랑교회 신도 9명 광주 57·58·59·60·61·62·63·68·69번 은 이날 코로나19 진단검사 결과 양성 판정을 받았다. 이 교회에서 광주 46번 확진자가 나오자 방역당국에서 접촉자 진단검사를 실시한 결과다. 더구나 광주 46번 확진자가 요양보호사로 일하는 광주 동구 동명동 C